In [1]:
#
# a program to make .fsgd file for fREEsURFER-script fs_glm.sh from .xlsx files
#
import pandas as pd
import os
IMPACT_CWD=os.path.expanduser("~")+"/Downloads/impact/"
#print(IMPACT_CWD)
bl_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_01_Access用_20200108.xlsx') # base lien file
fu_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_02_Access用_20200108.xlsx') # follow up file

# 手動で取り除くIDを指定
IDsQCfailed=["MR0011_0742","MR0011_0770","MR0011_0810"]

In [2]:
# read base line
bl_sheet_name = bl_book.sheet_names
bl_sheet_df = bl_book.parse(bl_sheet_name[0],skiprows=3,skipfooter=5)
bl_sheet_df=bl_sheet_df[bl_sheet_df[bl_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", bl_sheet_name[0])
bl_sheet_df=bl_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01","Gender_12_01","MRI_Imaging_Date_01",\
                               "FABHQ_01","FABHQ_DMN_01","FABHQ_CEN_01","FABHQ_SN_01",\
                               "GMBHQ_01","GMBHQ_DMN_01","GMBHQ_CEN_01","GMBHQ_SN_01",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_01","TMT_PartB_01_01",\
                               "MMSEJ_SUM_01","CESD_Score_01"]] # only base lines have MMSE
#print(bl_sheet_df.shape)
#print(bl_sheet_df.tail(5))

# read follow up
fu_sheet_name = fu_book.sheet_names
fu_sheet_df=fu_book.parse(fu_sheet_name[0],skiprows=3,skipfooter=5)
fu_sheet_df=fu_sheet_df[fu_sheet_df[fu_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", fu_sheet_name[0])
fu_sheet_df=fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_02","Age_02","Gender_12_02","MRI_Imaging_Date_02",\
                               "FABHQ_02","FABHQ_DMN_02","FABHQ_CEN_02","FABHQ_SN_02",\
                               "GMBHQ_02","GMBHQ_DMN_02","GMBHQ_CEN_02","GMBHQ_SN_02",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_02","TMT_PartB_01_02"]] #,"MMSEJ_SUM_01","CESD_Score_01"
#print(fu_sheet_df.shape)
#print(fu_sheet_df.tail(5))

# merge baseline and followup
bl_fu_sheet_df=pd.merge(bl_sheet_df, fu_sheet_df, left_on=bl_sheet_df.columns[0], right_on=fu_sheet_df.columns[0], how="inner", suffixes = ["_bl", "_fu"])
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace(" ","_")
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace("\n","_")
"""
ImPACT_ID	MRI_ID_01	Arm_Type_01	Arm_Number_01	Age_01	Gender_MF_01	Gender_12_01	MRI_Imaging_Date_01	MRI_Imaging_Time_01	
Height_01	Weight_01	Systolic_BP_01	Diastolic_BP_01	Pulse_01	
FABHQ_01	FABHQ_DMN_01	FABHQ_CEN_01	FABHQ_SN_01	GMBHQ_01	GMBHQ_DMN_01	GMBHQ_CEN_01	GMBHQ_SN_01	
SFC_Raw_01_01	SFC_Raw_02_01	SFC_Raw_03_01	SFC_Raw_04_01	SFC_Raw_05_01	SFC_Raw_06_01	SFC_Raw_07_01	SFC_Raw_08_01	SFC_Raw_09_01	SFC_Raw_10_01	SFC_Raw_11_01	SFC_Raw_12_01	SFC_Raw_13_01	SFC_Raw_14_01	SFC_Raw_15_01	SFC_Raw_16_01	SFC_Raw_17_01	SFC_Raw_18_01	SFC_Raw_19_01	SFC_Raw_20_01	
SFC_Physical_01	SFC_Mental_01	SFC_Score_01	
CESD_Raw_01_01	CESD_Raw_02_01	CESD_Raw_03_01	CESD_Raw_04_01	CESD_Raw_05_01	CESD_Raw_06_01	CESD_Raw_07_01	CESD_Raw_08_01	CESD_Raw_09_01	CESD_Raw_10_01	CESD_Raw_11_01	CESD_Raw_12_01	CESD_Raw_13_01	CESD_Raw_14_01	CESD_Raw_15_01	CESD_Raw_16_01	CESD_Raw_17_01	CESD_Raw_18_01	CESD_Raw_19_01	CESD_Raw_20_01	
CESD_Score_01_01	CESD_Score_02_01	CESD_Score_03_01	CESD_Score_04_01	CESD_Score_05_01	CESD_Score_06_01	CESD_Score_07_01	CESD_Score_08_01	CESD_Score_09_01	CESD_Score_10_01	CESD_Score_11_01	CESD_Score_12_01	CESD_Score_13_01	CESD_Score_14_01	CESD_Score_15_01	CESD_Score_16_01	CESD_Score_17_01	CESD_Score_18_01	CESD_Score_19_01	CESD_Score_20_01	
CESD_Score_01	CESD_T_01	
STAI_Raw_01_01	STAI_Raw_02_01	STAI_Raw_03_01	STAI_Raw_04_01	STAI_Raw_05_01	STAI_Raw_06_01	STAI_Raw_07_01	STAI_Raw_08_01	STAI_Raw_09_01	STAI_Raw_10_01	STAI_Raw_11_01	STAI_Raw_12_01	STAI_Raw_13_01	STAI_Raw_14_01	STAI_Raw_15_01	STAI_Raw_16_01	STAI_Raw_17_01	STAI_Raw_18_01	STAI_Raw_19_01	STAI_Raw_20_01	STAI_Raw_21_01	STAI_Raw_22_01	STAI_Raw_23_01	STAI_Raw_24_01	STAI_Raw_25_01	STAI_Raw_26_01	STAI_Raw_27_01	STAI_Raw_28_01	STAI_Raw_29_01	STAI_Raw_30_01	STAI_Raw_31_01	STAI_Raw_32_01	STAI_Raw_33_01	STAI_Raw_34_01	STAI_Raw_35_01	STAI_Raw_36_01	STAI_Raw_37_01	STAI_Raw_38_01	STAI_Raw_39_01	STAI_Raw_40_01	
STAI_Y1_P_01	STAI_Y1_A_01	STAI_Y2_P_01	STAI_Y2_A_01	STAI_Y1_SUM_01	STAI_Y2_SUM_01	STAI_Y1_T_01	STAI_Y2_T_01	STAI_Y1_5D_01	STAI_Y2_5D_01	
POMS2_Raw_01_01	POMS2_Raw_02_01	POMS2_Raw_03_01	POMS2_Raw_04_01	POMS2_Raw_05_01	POMS2_Raw_06_01	POMS2_Raw_07_01	POMS2_Raw_08_01	POMS2_Raw_09_01	POMS2_Raw_10_01	POMS2_Raw_11_01	POMS2_Raw_12_01	POMS2_Raw_13_01	POMS2_Raw_14_01	POMS2_Raw_15_01	POMS2_Raw_16_01	POMS2_Raw_17_01	POMS2_Raw_18_01	POMS2_Raw_19_01	POMS2_Raw_20_01	POMS2_Raw_21_01	POMS2_Raw_22_01	POMS2_Raw_23_01	POMS2_Raw_24_01	POMS2_Raw_25_01	POMS2_Raw_26_01	POMS2_Raw_27_01	POMS2_Raw_28_01	POMS2_Raw_29_01	POMS2_Raw_30_01	POMS2_Raw_31_01	POMS2_Raw_32_01	POMS2_Raw_33_01	POMS2_Raw_34_01	POMS2_Raw_35_01	
POMS2_AH_01	POMS2_CB_01	POMS2_DD_01	POMS2_FI_01	POMS2_TA_01	POMS2_VA_01	POMS2_F_01	POMS2_TMD_01	POMS2_AH_T_01	POMS2_CB_T_01	POMS2_DD_T_01	POMS2_FI_T_01	POMS2_TA_T_01	POMS2_VA_T_01	POMS2_F_T_01	POMS2_TMD_T_01	
MMSEJ_Raw_01_01	MMSEJ_Raw_02_01	MMSEJ_Raw_03_01	MMSEJ_Raw_04_01	MMSEJ_Raw_05_01	MMSEJ_Raw_06_01	MMSEJ_Raw_07_01	MMSEJ_Raw_08_01	MMSEJ_Raw_09_01	MMSEJ_Raw_10_01	MMSEJ_Raw_11_01	MMSEJ_Raw_12_01	MMSEJ_Raw_13_01	MMSEJ_Raw_14_01	MMSEJ_Raw_15_01	MMSEJ_Raw_16_01	MMSEJ_Raw_17_01	MMSEJ_Raw_18_01	MMSEJ_Raw_19_01	MMSEJ_Raw_20_01	MMSEJ_Raw_21_01	MMSEJ_Raw_22_01	MMSEJ_Raw_23_01	MMSEJ_Raw_24_01	MMSEJ_Raw_25_01	MMSEJ_Raw_26_01	MMSEJ_Raw_27_01	MMSEJ_Raw_28_01	MMSEJ_Raw_29_01	MMSEJ_Raw_30_01	MMSEJ_Raw_31_01	MMSEJ_Raw_32_01	MMSEJ_Raw_33_01	MMSEJ_Raw_34_01	MMSEJ_Raw_35_01	
MMSEJ_SUM_01	
SPA_Related_Raw_01_01	SPA_Related_Raw_02_01	SPA_Related_Raw_03_01	SPA_Unrelated_Raw_01_01	SPA_Unrelated_Raw_02_01	SPA_Unrelated_Raw_03_01	
SPA_Unrelated_SUM_01	SPA_Rank_01	
Stroop_01_01	Stroop_02_01	Stroop_03_01	Stroop_Score_01	
TMT_PartA_01_01	TMT_PartA_02_01	TMT_PartA_03_01	TMT_PartA_04_01	
TMT_PartB_01_01	TMT_PartB_02_01	TMT_PartB_03_01	TMT_PartB_04_01	Memo	END
"""

'\nImPACT_ID\tMRI_ID_01\tArm_Type_01\tArm_Number_01\tAge_01\tGender_MF_01\tGender_12_01\tMRI_Imaging_Date_01\tMRI_Imaging_Time_01\t\nHeight_01\tWeight_01\tSystolic_BP_01\tDiastolic_BP_01\tPulse_01\t\nFABHQ_01\tFABHQ_DMN_01\tFABHQ_CEN_01\tFABHQ_SN_01\tGMBHQ_01\tGMBHQ_DMN_01\tGMBHQ_CEN_01\tGMBHQ_SN_01\t\nSFC_Raw_01_01\tSFC_Raw_02_01\tSFC_Raw_03_01\tSFC_Raw_04_01\tSFC_Raw_05_01\tSFC_Raw_06_01\tSFC_Raw_07_01\tSFC_Raw_08_01\tSFC_Raw_09_01\tSFC_Raw_10_01\tSFC_Raw_11_01\tSFC_Raw_12_01\tSFC_Raw_13_01\tSFC_Raw_14_01\tSFC_Raw_15_01\tSFC_Raw_16_01\tSFC_Raw_17_01\tSFC_Raw_18_01\tSFC_Raw_19_01\tSFC_Raw_20_01\t\nSFC_Physical_01\tSFC_Mental_01\tSFC_Score_01\t\nCESD_Raw_01_01\tCESD_Raw_02_01\tCESD_Raw_03_01\tCESD_Raw_04_01\tCESD_Raw_05_01\tCESD_Raw_06_01\tCESD_Raw_07_01\tCESD_Raw_08_01\tCESD_Raw_09_01\tCESD_Raw_10_01\tCESD_Raw_11_01\tCESD_Raw_12_01\tCESD_Raw_13_01\tCESD_Raw_14_01\tCESD_Raw_15_01\tCESD_Raw_16_01\tCESD_Raw_17_01\tCESD_Raw_18_01\tCESD_Raw_19_01\tCESD_Raw_20_01\t\nCESD_Score_01_01\tCESD_S

In [3]:
# remove IDs whose QC failed
bl_fu_sheet_df=bl_fu_sheet_df[~bl_fu_sheet_df["ImPACT_ID"].isin(IDsQCfailed)]

# remove IDs depressed
"""
print("IDs depressed:")
print(bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]>=16][["ImPACT_ID","Arm_Type_01","CESD_Score_01"]])
"""
bl_fu_sheet_df=bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]<16]

# make data frame
df_result=bl_fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01","Gender_12_01","MRI_Imaging_Date_01",\
                               "SPA_Unrelated_SUM_01_bl","TMT_PartA_01_01","TMT_PartB_01_01",\
                               "MMSEJ_SUM_01",\
                                          "Age_02","MRI_Imaging_Date_02",\
                               "SPA_Unrelated_SUM_01_fu","TMT_PartA_01_02","TMT_PartB_01_02"]]
df_result["fID"]="V_"+df_result["ImPACT_ID"]+"_"+df_result["Arm_Type_01"]
#print(df_result.shape)
#print(df_result.head(2))
#print(type(df_result))
df_result=df_result.drop(columns=["Age_02"]) # work around for coincidence
df_result=df_result.T.drop_duplicates().T # 重複列削除
df_result=df_result.drop(columns=["MRI_Imaging_Date_01", "MRI_Imaging_Date_02"])
df_result=df_result.set_index("fID")
#print(df_result.shape)
#print(df_result.head(2))
#print(type(df_result))

In [4]:
# make .fsgd file for fREEsURFER-script fs_glm.sh
impact_grouplist=["AR","AS","AT","AU"]
ctrl_grouplist=["AR"]
intrv_grouplist=["AS","AT","AU"]

# 年齢と相関する皮質厚
for ss in impact_grouplist:
    filename=IMPACT_CWD+"g1v1_"+ss+".fsgd"
    outfile=open(filename, "wt")
    outfile.write("GroupDescriptorFile 1"+"\n")
    outfile.write("Title"+" "+"age_"+ss+"\n")
    outfile.write("MeasurementName thickness"+"\n")
    outfile.write("Class Main"+"\n")
    outfile.write("Variables"+" "+"age"+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AR_01"+" "+"Main"+" "+str(50))
    outfile.close()

filename=IMPACT_CWD+"g1v1_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(0)+" "+str(1))
outfile.close()

# 反応時間と相関する皮質厚
for ss in impact_grouplist:
    filename=IMPACT_CWD+"g1v1_score_"+ss+".fsgd"
    outfile=open(filename, "wt")
    outfile.write("GroupDescriptorFile 1"+"\n")
    outfile.write("Title"+" "+"score_"+ss+"\n")
    outfile.write("MeasurementName thickness"+"\n")
    outfile.write("Class Main"+"\n")
    outfile.write("Variables"+" "+"age"+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AR_01"+" "+"Main"+" "+str(50)+"\n")
    outfile.close()

# usage:
# from Terminal$ call fREEsURFER-scripts as follows
# Terminal$ fs_glm.sh $(IMPACT_CWD+"g1v1_"+$ss+".fsgd") $(IMPACT_CWD+"g1v1_common.mtx")
# Terminal$ fs_view_glm.sh $("lh.age_corr_"+$ss+".thickness.10.glmdir") $(IMPACT_CWD+"g1v1_common.mtx") 1 4
# Terminal$ fs_view_glm_allfwhm.sh $("lh.age_corr_"+$ss+".thickness.10.glmdir") $(IMPACT_CWD+"g1v1_common.mtx") 1 4

# 年齢調整後の反応時間と相関する皮質厚
for ss in impact_grouplist:
    filename=IMPACT_CWD+"g1v2_"+ss+".fsgd"
    outfile=open(filename, "wt")
    outfile.write("GroupDescriptorFile 1"+"\n")
    outfile.write("Title"+" "+"age_score_"+ss+"\n")
    outfile.write("MeasurementName thickness"+"\n")
    outfile.write("Class Main"+"\n")
    outfile.write("Variables"+" "+"age"+" "+"score"+"\n")
    """
    SPA_Unrelated_SUM_01_bl
    TMT_PartA_01_01
    TMT_PartB_01_01
    MMSEJ_SUM_01
    SPA_Unrelated_SUM_01_fu
    TMT_PartA_01_02
    TMT_PartB_01_02
    """
    outfile.write("Input"+" "+"V_MR0011_0653_AR_01"+" "+"Main"+" "+str(50)+" "+str(50)+"\n")
    outfile.close()

filename=IMPACT_CWD+"g1v2_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(0)+" "+str(0)+" "+str(1))
outfile.close()

# usage:
# from Terminal$ call fREEsURFER-scripts as follows
# fs_glm.sh g1v2agerestime.fsgd g1v2_v.mtx
# fs_view_glm.sh lh.agerestime_corr.thickness.10.glmdir  g1v2_v.mtx 1 4
# fs_view_glm_allfwhm.sh lh.agerestime_corr.thickness.10.glmdir g1v2_v.mtx 
# 
# 多重比較補正 CDT=1.3(p＝0.05)は不適。実際にはCDT=3.0(p=0.001)で行えば良い。
# fs_glmfit-sim.sh lh.agerestime_corr.thickness.10.glmdir 3.0
# freeview -f $SUBJECTS_DIR/fsaverage/surf/lh.inflated:overlay=lh.agerestime_corr.thickness.10.glmdir/g1v2_v/cache.th13.pos.sig.cluster.mgh:annot=lh.agerestime_corr.thickness.10.glmdir/g1v2_v/cache.th13.pos.sig.ocn.annot  -layout 1 -viewport 3d

# g2v0: 
for ss in intrv_grouplist:
    filename=IMPACT_CWD+"g2v1_"+ss+".fsgd"
    outfile=open(filename, "wt")
    outfile.write("GroupDescriptorFile 1"+"\n")
    outfile.write("Title"+" "+"group_"+ss+"\n")
    outfile.write("MeasurementName thickness"+"\n")
    outfile.write("Class "+ctrl_grouplist[0]+"\n")
    outfile.write("Class "+ss+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AR_01"+" "+ctrl_grouplist[0]+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AV_01"+" "+ss+"\n")
    outfile.close()

filename=IMPACT_CWD+"g2v0_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(1)+" "+str(-1))
outfile.close()

# g2v1: 年齢調整後のgroup別の皮質厚の差
# g2v1_gx: group別で年齢と皮質厚の相関に違いがある領域
for ss in intrv_grouplist:
    filename=IMPACT_CWD+"g2v1_"+ss+".fsgd"
    outfile=open(filename, "wt")
    outfile.write("GroupDescriptorFile 1"+"\n")
    outfile.write("Title"+" "+"group_age_"+ss+"\n")
    outfile.write("MeasurementName thickness"+"\n")
    outfile.write("Class "+ctrl_grouplist[0]+"\n")
    outfile.write("Class "+ss+"\n")
    outfile.write("Variables"+" "+"age"+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AR_01"+" "+ctrl_grouplist[0]+" "+str(64)+"\n")
    outfile.write("Input"+" "+"V_MR0011_0653_AV_01"+" "+ss+" "+str(64)+"\n")
    outfile.close()

filename=IMPACT_CWD+"g2v1_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(1)+" "+str(-1)+" "+str(0)+" "+str(0))
outfile.close()

filename=IMPACT_CWD+"g2v1_gx_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(0)+" "+str(0)+" "+str(1)+" "+str(-1))
outfile.close()